# `libcusmm`: explore the space of autotuned parameters

This notebook can be used to explore the space of autotuned parameters, stored in files named `parameters_GPU.json`.

## Library imports

In [ ]:
import numpy as np
import pandas as pd
import json, os

## Read data

#### Choose a GPU

In [ ]:
GPU = 'P100' # Options: K20X, K40, K80, P100, V100, Mi50

In [ ]:
params = '../parameters_' + GPU + '.json'  
assert os.path.exists(params)

#### Read autotuned parameters

In [ ]:
with open(params) as f:
    all_parameters = pd.DataFrame([params for params in json.load(f)])
autotuned_parameters = all_parameters[all_parameters['source'] == 'autotuned']
print("Reading autotuned data from", params)

In [ ]:
ordered_columns = ['m', 'n', 'k', 'perf', 'algorithm', 'threads', 'grouping', 'minblocks', 'tile_m', 'tile_n', 'v', 'w']
autotuned_parameters = autotuned_parameters[ordered_columns]
print('Autotuned parameters:')
display(autotuned_parameters)

## Data Description

In [ ]:
print('Numer of columns:', len(autotuned_parameters.columns), '\nNumber of rows:', len(autotuned_parameters.index.values))
print('\nColumn names:')
for c in autotuned_parameters.columns.values: 
    print(c)

In [ ]:
autotuned_parameters.describe()

In [ ]:
import pandas_profiling 
pandas_profiling.ProfileReport(autotuned_parameters)

## Plot performances

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt

In [ ]:
autotuned_parameters['mnk'] = autotuned_parameters['m'] * autotuned_parameters['n'] * autotuned_parameters['k']
plt.semilogx(autotuned_parameters['mnk'], autotuned_parameters['perf'], '.', markersize=3)
plt.xlabel('Training (m, n, k) triplets (in order of increasing m*n*k)')
plt.ylabel('Performance [Gflops]')

## Parameter frequencies

In [ ]:
# ignore the 'threads' parameter since it has to be adapted to the size of matrix C
parameter_set = ['algorithm', 'grouping', 'minblocks', 'tile_m', 'tile_n', 'v', 'w']

### Most frequent parameter sets

In [ ]:
def get_par_set(algorithm, grouping, minblocks, tile_m, tile_n, v, w):
    par_set= algorithm + '_' + str(int(grouping)) + '_' + str(int(minblocks)) 
    if not np.isnan(tile_m):
        par_set +=  '_' + str(int(tile_m)) + '_' + str(int(tile_n))
        if not np.isnan(v):
            par_set += '_' + str(int(v)) + '_' + str(int(w))
    return par_set
    
vget = np.vectorize(get_par_set)
autotuned_parameters['param_set'] = vget(*[a for a in autotuned_parameters[parameter_set].values.transpose()])

In [ ]:
param_set_freq = autotuned_parameters['param_set'].value_counts(dropna=True)
autotuned_parameters['param_set_freq'] = autotuned_parameters['param_set'].apply(lambda item: param_set_freq[item])
autotuned_parameters.sort_values(by='param_set_freq', ascending=False, inplace=True)
autotuned_parameters.iloc[:50,:]

### Most frequent parameters (independently of each other)

In [ ]:
most_frequent_values = dict()
for c in autotuned_parameters.columns.values: 
    plt.figure
    plt.hist(autotuned_parameters[c].dropna(), bins=50)
    plt.title(c)
    plt.show()
    if c in parameter_set: 
        col = autotuned_parameters[c].dropna().values
        values, counts = np.unique(col, return_counts=True)
        ind_most_freq = np.argmax(counts)
        most_freq_val = values[ind_most_freq]
        most_frequent_values[c] = most_freq_val